In [25]:
import requests
from lxml import html
import re
import csv

#解析html网页
def parse_tree(url):
    r = requests.get(url)
    tree = html.fromstring(r.text)
    return tree

In [55]:
#获取title
def get_title(tree):
    titles = tree.xpath('//*[@id="content"]/div/div[1]/ol/*/div/div[2]/div[1]/a/span[1]/text()')
    arr_title = []
    # print(titles)
    for title in titles:
        arr_title.append(str(title))
#     print(arr_title)
    return arr_title

In [56]:
#获取导演和主演，只取第一个名字
def get_da(tree):
    doctor = tree.xpath('//*[@id="content"]/div/div[1]/ol/*/div/div[2]/div[2]/p[1]/text()[1]')
    da = []
    for d in doctor:
#         print(d)
        dd = re.split(r'[\s\\xa0]', str(d))
#         print(dd)
        dactor = ''
        actor = ''
        for s in dd:
            if '导演' in s:
                index1 = dd.index(s)
                if index1+1 < len(dd):
                    dactor = dd[index1+1]
            elif '主演' in s:
                index = dd.index(s)
                if index+1 < len(dd):
                    actor = dd[index+1]
                break
        arr = [dactor, actor]
        da.append(arr)
#     print(da)
    return da

In [57]:
#获取年份
def get_years(tree):
    years = tree.xpath('//*[@id="content"]/div/div[1]/ol/*/div/div[2]/div[2]/p[1]/text()[2]')
    arr_year = []
    for y in years:
        year = str(y).strip('\n').strip().split('\xa0/\xa0')
        arr_year.append(year)
#     print(arr_year)
    return arr_year

In [58]:
#获取评分
def get_rate(tree):
    rates = tree.xpath('//*[@id="content"]/div/div[1]/ol/*/div/div[2]/div[2]/div/span[2]/text()')
    arr_rate = []
    # print(rates)
    for r in rates:
        arr_rate.append(str(r))
#     print(arr_rate)
    return arr_rate

In [59]:
#将结果拼接到result中
def append(result,arr_title, arr_da, arr_year, arr_rate):
    for i in range(len(arr_title)):
        a = (arr_title[i], arr_da[i][0],arr_da[i][1], arr_year[i][0],arr_year[i][1],arr_year[i][2], arr_rate[i])
        result.append(a)



In [68]:
result_title = ['名字', '导演', '主演', '年份', '国家', '类型', '评分']
result = []

for i in range(10): #一共10页
    count = i * 25 #每页25条数据
    url = 'https://movie.douban.com/top250?start=%d' %(count) #拼接每页的url地址
    print(url)
    tree = parse_tree(url) #解析html
    #拼接数据到result中
    append(result,get_title(tree), get_da(tree), get_years(tree), get_rate(tree))

#输出一下看看结果
print(result_title)
for r in result:
    print(r) 

#保存到csv中， newline字段是设置换行，默认是'\n'，会直接在csv中插入空行
csvfile = open('douban_top250.csv', 'w', encoding='gb18030', newline='')
writer = csv.writer(csvfile)
#先写入title
writer.writerow(result_title)
print('====================')
for r in result:
    print(r)
    #写入每一条数据，如果出错则跳过该条数据
    try:
        writer.writerow(r)
    except:
        continue
csvfile.close() #最后关闭文件

https://movie.douban.com/top250?start=0
[['弗兰克·德拉邦特', '蒂姆·罗宾斯'], ['陈凯歌', '张国荣'], ['吕克·贝松', '让·雷诺'], ['Robert', 'Tom'], ['罗伯托·贝尼尼', '罗伯托·贝尼尼'], ['宫崎骏', '柊瑠美'], ['史蒂文·斯皮尔伯格', '连姆·尼森'], ['詹姆斯·卡梅隆', '莱昂纳多·迪卡普里奥'], ['克里斯托弗·诺兰', '莱昂纳多·迪卡普里奥'], ['安德鲁·斯坦顿', '本·贝尔特'], ['朱塞佩·托纳多雷', '蒂姆·罗斯'], ['拉库马·希拉尼', '阿米尔·汗'], ['莱塞·霍尔斯道姆', '理查·基尔'], ['克里斯托夫·巴拉蒂', '杰拉尔·朱诺'], ['刘镇伟', '周星驰'], ['弗朗西斯·福特·科波拉', '马龙·白兰度'], ['宫崎骏', '日高法子'], ['彼得·威尔', '金·凯瑞'], ['维克多·弗莱明', '托...'], ['黄东赫', '孔侑'], ['奥利维·那卡什', ''], ['朱塞佩·托纳多雷', '安东娜拉·塔莉'], ['加布里尔·穆奇诺', '威尔·史密斯'], ['刘伟强', '刘德华'], ['克里斯托弗·诺兰', '马修·麦康纳']]
https://movie.douban.com/top250?start=25
[['罗伯·莱纳', '玛德琳·卡罗尔'], ['Sidney', '亨利·方达'], ['大卫·芬奇', '爱德华·诺顿'], ['李安', '苏拉·沙玛'], ['彼得·杰克逊', '维果·莫腾森'], ['姜文', '姜文'], ['克里斯托弗·诺兰', '克里斯蒂安·贝尔'], ['宫崎骏', '田中真弓'], ['张艺谋', '葛优'], ['威廉·惠勒', '奥黛丽·赫本'], ['刘镇伟', '周星驰'], ['彼特·道格特', '爱德...'], ['Guy', 'J'], ['弗洛里安·亨克尔·冯·多纳斯马', ''], ['米洛斯·福尔曼', '杰克·尼科尔森'], ['比利·怀尔德', '查尔斯·劳顿'], ['Louie', 'John'], ['拜伦·霍华德', '金妮弗·...'], ['马丁·布莱斯', '阿尔·帕西诺'], ['